In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
cidades = pd.read_excel('BRCidadesRegiao.xlsx')
cidades

In [ ]:
cidades = cidades[['regiao_imediata', 'cidade', 'estado', 'longitude', 'latitude', 'altitude',
'area_territorial', 'populacao_estimada', 'densidade_demografica', 'pib_per_capita', 'receitas_realizadas',
'despesas_empenhadas', 'saldo_receitas_despesas', 'unidades', 'unidades_urbanas', 'unidades_rurais', 'area_plantada', 'producao_graos',
'IDHM_Renda', 'IDHM_Longevidade', 'IDHM_Educacao', 'GVA_agropecuaria', 'GVA_industria', 'GVA_Servicos', 'GVA_publico',
'GDP', 'GDP_populacao', 'GDP_per_capita', 'estabelecimentos', 'carros', 'motos'
]].copy()

In [ ]:
cidades.info()

In [ ]:
cidades.replace([np.inf, -np.inf], np.nan, inplace=True)
cidades.dropna(inplace=True)
cidades

In [ ]:
cidades['estado'].unique()

In [ ]:
cidades_SC = cidades[cidades['estado'] == 'SC'].copy()
cidades_SC

In [ ]:
vif_dt = cidades_SC[[
'area_territorial', 'populacao_estimada', 'densidade_demografica', 'pib_per_capita', 'receitas_realizadas',
'despesas_empenhadas', 'saldo_receitas_despesas', 'unidades', 'unidades_urbanas', 'unidades_rurais', 'area_plantada', 'producao_graos',
'IDHM_Renda', 'IDHM_Longevidade', 'IDHM_Educacao', 'GVA_agropecuaria', 'GVA_industria', 'GVA_Servicos', 'GVA_publico',
'GDP', 'GDP_populacao', 'GDP_per_capita', 'estabelecimentos', 'carros', 'motos'
]]
vif_dt

In [ ]:
vif_data = pd.DataFrame()
vif_data['variavel'] = vif_dt.columns
vif_data['VIF'] = [variance_inflation_factor(vif_dt.values, i) 
    for i in range(len(vif_dt.columns))]
round(vif_data,4)

In [ ]:
vif_dt = cidades_SC[[
'area_territorial', 'populacao_estimada', 'pib_per_capita', 
'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP']]
vif_dt

In [ ]:
vif_data = pd.DataFrame()
vif_data['variavel'] = vif_dt.columns
vif_data['VIF'] = [variance_inflation_factor(vif_dt.values, i) 
    for i in range(len(vif_dt.columns))]
vif_data

In [ ]:
vif_dt = vif_dt.copy()
for coluna in vif_dt.columns:
    print(coluna)
    out_max = vif_dt[coluna].quantile(0.75) + ((vif_dt[coluna].quantile(0.75) - vif_dt[coluna].quantile(0.25)) * 1.5)
    print(out_max)
    vif_dt[coluna] = vif_dt[coluna].apply(lambda x : out_max if x > out_max else x)
    out_min = vif_dt[coluna].quantile(0.25) - ((vif_dt[coluna].quantile(0.75) - vif_dt[coluna].quantile(0.25)) * 1.5)
    print(out_min)
    vif_dt[coluna] = vif_dt[coluna].apply(lambda x : out_min if x < out_min else x)
vif_dt

In [ ]:
colunas = ['area_territorial', 'populacao_estimada', 'pib_per_capita', 'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP']
sel = colunas
cidadesz = vif_dt[sel].copy()
listaz = []
for coluna in colunas:
    if coluna != "cidade":
        print(coluna)
        listaz.insert(0, 'z_'+coluna)
        media = cidadesz[coluna].mean()
        dp = cidadesz[coluna].std()
        cidadesz['z_'+coluna] = cidadesz[coluna].apply(lambda x : (x - media) / dp)
cidadesz['totalz'] = cidadesz[listaz].sum(axis=1)
cidadesz['ranking'] = cidadesz['totalz'].rank(ascending=False)
cidadesz   

In [ ]:
cidades_SC = cidades_SC.merge(cidadesz, right_index=True, left_index=True, how = 'left')
cidades_SC

In [ ]:
cidades_SC.sort_values(by='ranking')[1:10]

In [ ]:
cidades_SC.sort_values(by='ranking', ascending=False)[1:10]

In [ ]:
regioes = cidades.groupby(['estado','regiao_imediata'])[['area_territorial', 'populacao_estimada', 'pib_per_capita', 'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP']].mean().reset_index().copy()
regioes

In [ ]:
regioes_SC = regioes[regioes['estado'] == 'SC']
regioes_SC

In [ ]:
colunas = ['area_territorial', 'populacao_estimada', 'pib_per_capita', 'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP']
sel = colunas
regioesz = regioes_SC[sel].copy()
listaz = []
for coluna in colunas:
    if (coluna != "estado") & (coluna != "regiao_imediata"):
        print(coluna)
        listaz.insert(0, 'z_'+coluna)
        media = regioesz[coluna].mean()
        dp = regioesz[coluna].std()
        regioesz['z_'+coluna] = regioesz[coluna].apply(lambda x : (x - media) / dp)
regioesz['totalz'] = regioesz[listaz].sum(axis=1)
regioesz['ranking'] = regioesz['totalz'].rank(ascending=False)
regioesz  

In [ ]:
regioes_SC = regioes_SC.merge(regioesz, right_index=True, left_index=True, how = 'left')
regioes_SC

In [ ]:
regioes_SC.sort_values(by='ranking')[1:4]

In [ ]:
regioes_SC.sort_values(by='ranking', ascending=False)[1:4]